# MNIST handwritten digits clustering

In this notebook, we'll use some common clustering algorithms to analyze MNIST digits using scikit-learn.

First, the needed imports. 

In [ ]:
%matplotlib inline

from pml_utils import get_mnist, show_clusters

import numpy as np
from sklearn import __version__
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from distutils.version import LooseVersion as LV
assert(LV(__version__) >= LV("0.20")), "Version >= 0.20 of sklearn is required."

Then we load the MNIST data. First time it downloads the data, which can take a while.

To speed up the computations, let's use only 10000 digits in this notebook.

In [ ]:
X_train, y_train, X_test, y_test = get_mnist('MNIST')

X = X_train[:10000]
y = y_train[:10000]
print()
print('MNIST data loaded:')
print('X:', X.shape)
print('y:', y.shape)

## k-means

[K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) clusters data by trying to separate samples in *k* groups of equal variance using an iterative two-step algorithm. It requires the number of clusters as a parameter.

In [ ]:
%%time

n_clusters_kmeans = 10

kmeans = KMeans(n_clusters=n_clusters_kmeans)
kmeans.fit(X)

The sizes of the clusters:

In [ ]:
plt.hist(kmeans.labels_, bins=range(kmeans.n_clusters+1),
         rwidth=0.5)
plt.xticks(0.5+np.arange(kmeans.n_clusters),
           np.arange(kmeans.n_clusters))
plt.title('Cluster sizes');

The k-means centroids are vectors in the same space as the original data, so we can take a look at them:

In [ ]:
plt.figure(figsize=(kmeans.n_clusters, 1))

for i in range(kmeans.n_clusters):
    plt.subplot(1, kmeans.n_clusters, i+1)
    plt.axis('off')
    plt.imshow(kmeans.cluster_centers_[i,:].reshape(28,28), cmap="gray")
    plt.title(str(i))

Let's also draw some digits from each cluster:

In [ ]:
show_clusters(kmeans.labels_, kmeans.n_clusters, X)

### Evaluation

Since we know the correct labels for MNIST digits, we can evaluate the quality of the clustering. We'll use the [adjusted Rand index](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html) which considers all pairs of samples and counts pairs that are assigned in the same or different clusters in the predicted and true clusterings. The index is between 0.0 and 1.0 with higher values denoting better clusterings.

In [ ]:
print("Adjusted Rand index: %.3f"
      % adjusted_rand_score(y, kmeans.labels_))

## Hierarchical clustering

Hierarchical clustering is a family of clustering algorithms that build nested clusters by merging or splitting them successively. We'll use here [agglomerative clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering), in which all samples start in separate clusters and are then successively merged together.

The `linkage` criteria determines the metric used for the merge strategy:
* `ward` minimizes the sum of squared differences within all clusters
* `complete` linkage minimizes the maximum distance between observations of pairs of clusters
* `average` linkage minimizes the average of the distances between all observations of pairs of clusters
* `single` linkage minimizes the distance between the closest observations of pairs of clusters

In [ ]:
%%time

n_clusters_hclust = 10
linkage_hclust = "ward"

hclust = AgglomerativeClustering(n_clusters=n_clusters_hclust,
                                 linkage=linkage_hclust)
hclust.fit(X)

The sizes of the clusters:

In [ ]:
plt.hist(hclust.labels_, bins=range(hclust.n_clusters+1),
         rwidth=0.5)
plt.xticks(0.5+np.arange(hclust.n_clusters),
           np.arange(hclust.n_clusters))
plt.title('Cluster sizes');

Some digits from each cluster:

In [ ]:
show_clusters(hclust.labels_, hclust.n_clusters, X)

### Evaluation

In [ ]:
print("Adjusted Rand index: %.3f"
      % adjusted_rand_score(y, hclust.labels_))

## Experiments

1. Experiment with different numbers of clusters, different linkage criteria, and other parameters for [k-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) and [hierarchical clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering).

2. Evaluate different clustering methods using Rand index scores with `n_clusters=10`. What is the best clustering algorithm according to this measure?

3. Try other clustering methods available in scikit-learn. See [Clustering](https://scikit-learn.org/stable/modules/clustering.html#clustering) to get started.